# 01. Preprocessing Street View Housing Numbers (SVHN) Dataset

### Purpose:
Convert the annotations provided with the SVHN dataset to the Darknet TXT format.

### Before Running Notebook:
1. Download train.tar.gz and test.tar.gz from [here](http://ufldl.stanford.edu/housenumbers/).
1. Extract the files into the Data folder at the top level of this repository.
1. The relative path from this notebook to the extracted train and test files must be assigned to the path variable in the final two cells.
1. The `digitStruct.mat` file must be included in the same directory as the images that it describes.

### Details:
The annotations for the SVHN dataset are provided as an h5py formatted file named `digitStruct.mat`.  There is one file for train and one for test.  In this project, I will be using Darknet to train custom Yolo models with this dataset, and Darknet requires annotations to be separated into one text file per image.  

More information about Darknet annotation format can be found [here](https://github.com/AlexeyAB/darknet#how-to-train-to-detect-your-custom-objects).


In [1]:
import h5py
import cv2 as cv

## Defining Functions
---

In [3]:
def get_img_name(f, idx=0):
    """
    Get the name of an image given it's index.
    Adapted from https://www.vitaarca.net/post/tech/access_svhn_data_in_python/

    Args
        f: digitStruct.mat h5py file
        idx: index of the image
        
    Returns:
        image name as a string
    """
    names = f['digitStruct/name']
    img_name = ''.join(map(chr, f[names[idx][0]][()].flatten()))
    return(img_name)

In [4]:
def get_img_boxes(f, idx=0):
    """
    Get the 'height', 'left', 'top', 'width', 'label' of bounding boxes of an image
    Adapted from https://www.vitaarca.net/post/tech/access_svhn_data_in_python/

    Args
        f: digitStruct.mat h5py file
        idx: index of the image
        
    Returns:
        dictionary of bounding box values as integers
    """
    bboxs = f['digitStruct/bbox']
    box = f[bboxs[idx][0]]
    meta = { key : [] for key in box.keys()}

    for key in box.keys():
        if box[key].shape[0] == 1:
            meta[key].append(int(box[key][0][0]))
        else:
            for i in range(box[key].shape[0]):
                meta[key].append(int(f[box[key][i][0]][()].item()))

    return meta

In [6]:
def create_annot_file(f, path, idx=0):
    """
    Create a single Darknet TXT annotation file for an image.
    Writes to file <image name>.txt in same directory as image.

    Args
        f: digitStruct.mat h5py file
        path: path: path to digitStruct.mat
        idx: index of the image
        
    Returns:
        None
    """
    # get image name and bounding info
    name = get_img_name(f, idx)
    boxes = get_img_boxes(f, idx)
    
    # get dimensions of image
    try:
        (h_img, w_img) = cv.imread(path + name).shape[:2]
    except:
        print(f"ERROR: Could not open {name} to get dimensions.")
        print("Make sure image is in same directory as digitStruct.mat")
        print(f"Tried:  {path + name}")
        
    # initialize list for annotations
    annots = []
    
    for i in range(len(boxes['label'])):
        # get original bounding values
        (x, y) = (boxes['left'][i], boxes['top'][i])
        (w, h) = (boxes['width'][i], boxes['height'][i])

        # transform x and y
        centerX = x + (w / 2)
        centerY = y + (h / 2)

        # normalize bounding values
        centerX /= w_img
        centerY /= h_img
        w /= w_img
        h /= h_img

        # get label
        label = boxes['label'][i] if boxes['label'][i] != 10 else 0

        # append annotation in Darknet format to annotation list
        annots.append(f'{label} {centerX} {centerY} {w} {h}\n' )
    
    # write annotations to file 
    annot_file = open(path + name.split('.')[0] + '.txt', 'w')
    annot_file.writelines(annots)
    annot_file.close()

In [8]:
def create_annot_files(path):
    """
    Create Darknet TXT annotation file for all images in directory.
    Writes to files <image name>.txt in same directory as images.

    Args
        path: path to digitStruct.mat
        
    Returns:
        None
    """
    if path[-1] != '/':
        path += '/'
    
    try:
        f = h5py.File(f'{path}digitStruct.mat', mode='r')
    except:
        print("ERROR: Could not open file.  Check path to digitStruct.mat")
        
    for i in range(len(f['digitStruct/name'])):
        create_annot_file(f, path, i)

## Create Annotation Files
___

In [ ]:
# Create annotation files for train data
path = '../DATA/SVHN/Full/train'
create_annot_files(path)

In [ ]:
# Create annotation files for test data
path = '../DATA/SVHN/Full/test'
create_annot_files(path)

In [9]:
# Create annotation files for extra data
path = '../DATA/SVHN/Full/extra'
create_annot_files(path)